In [1]:
%load_ext nbtest

# Test Bank

This is an example test bank.

In [2]:
## Code shared by the test bank can go here.

def shared():
    pass

## The Simplest Question

Questions are specialized Python classes that include the question text, some information about what kind of solution is expected and test cases. To enable maximum reusability the variable `self.solution` will be defined by the framework code. In the simplest form, questions can expect:

1. The solution will be marked with a tag that has the same name as the test class (e.g. `ArgumentReverser`) 
2. `self.solution` will contain the `TagCacheEntry` with the solution cell. 

Here is a very simple test question:

In [3]:
from nbquiz import TestQuestion

class SimpleArgReverse(TestQuestion):
    """
    Write a function called `arg_reverser` that takes three arguments and returns
    them in reverse order.
    """

    def test_1_arg_reverser(self):
        """Testing arg_reverser()"""

        assert "arg_reverser" in self.solution.ns, """I can't find your solution."""
        self.assertEqual(self.solution.ns["arg_reverser"](1, 2, 3), (3, 2, 1))
        self.assertEqual(
            self.solution.ns["arg_reverser"]("one", "two", "three"), ("three", "two", "one")
        )

Generate a preview for copy/paste to an LMS

In [4]:
from IPython.display import Markdown
Markdown(SimpleArgReverse.question())



Write a function called `arg_reverser` that takes three arguments and returns
them in reverse order.


Add the tag `@SimpleArgReverse` to the docstring in your solution cell.


### Question Self Tests

To test the question you can check it against the correct and incorrect solutions:

In [5]:
"""@SimpleArgReverse"""

def arg_reverser(arg1, arg2, arg3):
    """Reverse my arguments."""
    return arg3, arg2, arg1

In [6]:
%%testing SimpleArgReverse
nbtest_cases = [SimpleArgReverse]

In [7]:
import nbtest
nbtest.assert_ok()

What about tests versus invalid solutions? They should be able to be in the notebook:

In [8]:
"""@SimpleArgReverse"""

def arg_reverser(a, b):
    """Wrong number of arguments."""
    return b, a

In [9]:
%%testing SimpleArgReverse
nbtest_cases = [SimpleArgReverse]

In [10]:
nbtest.assert_error()

Incorrect return values are reported:

In [11]:
"""@SimpleArgReverse"""

def arg_reverser(a, b, c):
    """Bad return value"""
    return b, a, c

In [12]:
%%testing SimpleArgReverse
nbtest_cases = [SimpleArgReverse]

In [13]:
nbtest.assert_error()

## Using FunctionQuestion 

The `FunctionQuestion` class supports a number of ways to validate a function question. Function questions can also become cell questions. See below. A function question:

1. Validates that the function exists and is a function. 
1. Ensures that the function has a docstring. 
1. Validates the number, order and names of arguments. 
1. Validates the return type by wrapping the student function. 

A simple function question is shown below:

In [14]:
import ast 
from nbquiz import FunctionQuestion

class TriangleMaxArea(FunctionQuestion):
    """
    Write a function called `triangle_limit` that takes two arguments, `base` and `height`. Calculate the area of a triangle with `base` and `height`. 
    If the area of the triangle is greater than or equal to `limit`, return `True` otherwise return `False`. 
    """

    tokens_required = [ast.GtE]
    tokens_forbidden = [ast.If]
    tags = []

    # self.solution will be this attribute
    name = "triangle_limit" 

    # Validate self.solution based on the type hints style attributes. 
    # When a FunctionValidator is used `self.solution` will be the function.
    annotations = {"base": float, "height": float, "return": bool}

    def test_the_func(self):
        self.assertEqual(
            self.solution(10, 10), True, """Should be True."""
        )
        self.assertEqual(
            self.solution(10,1), False, """Should be True."""
        )
        self.assertEqual(
            self.solution(10,0.1), False, """Should be False."""
        )


In [15]:
"""@TriangleMaxArea"""
def triangle_limit(base, height):
    """This is my solution."""
    return (base * height) / 2 >= 10

In [16]:
%%testing TriangleMaxArea
nbtest_cases = [TriangleMaxArea]

A number of error conditions are checked for:

In [17]:
"""@TriangleMaxArea"""

# Wrong or misspelled name. 
def triangle_limit_blah(base, height):
    """This is my solution."""
    return (base * height) / 2 >= 10

In [18]:
%%testing TriangleMaxArea
nbtest_cases = [TriangleMaxArea]

In [19]:
"""@TriangleMaxArea"""

# Parameters are incorrect or are in the wrong order:
def triangle_limit(height, base):
    """This is my solution."""
    return (base * height) / 2 >= 10

In [20]:
%%testing TriangleMaxArea
nbtest_cases = [TriangleMaxArea]

In [21]:
"""@TriangleMaxArea"""

# No docstring
def triangle_limit(base, height):
    return (base * height) / 2 >= 10

In [22]:
%%testing TriangleMaxArea
nbtest_cases = [TriangleMaxArea]

In [23]:
"""@TriangleMaxArea"""

# Wrong return type.
def triangle_limit(base, height):
    """This is my solution."""
    return (base >= height) / 2 

In [24]:
%%testing TriangleMaxArea
nbtest_cases = [TriangleMaxArea]

In [25]:
"""@TriangleMaxArea"""

# Missing the ast.GtE token.
def triangle_limit(base, height):
    """This is my solution."""
    return (base * height) / 2 <= 10

In [26]:
%%testing TriangleMaxArea
nbtest_cases = [TriangleMaxArea]

In [27]:
"""@TriangleMaxArea"""

# Uses the ast.If token
def triangle_limit(base, height):
    """This is my solution."""
    if (base * height) / 2 >= 10:
        return True
    else: 
        return False

In [28]:
%%testing TriangleMaxArea
nbtest_cases = [TriangleMaxArea]

## Parameterized Function Questions

Questions can be parameterized so that variants of the question can be created automatically. The previous version of `TriangleMaxArea` hard coded the function name, arguments and the constant `10`. This version parameterizes them using class variables. Care should be taken to ensure test cases work for all parameters. 

The special form of `{variable}` in the `annotations` dictionary and the docstring causes the validator to look up the function name in the class variable called `variable`. 

> NOTE: These are **not** f-strings. The `{variable}` is a literal.

In [29]:
import ast 
from nbquiz import FunctionQuestion

class TriangleMaxArea(FunctionQuestion):
    """
    Write a function called {name} that takes two arguments, {base} and {height}. Calculate the area of a triangle with {base} and {height}. 
    If the area of the triangle is greater than or equal to {limit}, return `True` otherwise return `False`. 
    """

    tokens_required = [ast.GtE]
    tokens_forbidden = [ast.If]
    tags = []

    # self.solution will be this attribute
    name = "triangle_limit" 
    base = "base"
    height = "height"
    limit = 10

    # Validate self.solution based on the type hints style attributes. 
    # When a FunctionValidator is used `self.solution` will be the function.
    annotations = {"{base}": float, "{height}": float, "return": bool}

    def test_the_func(self):
        self.assertEqual(
            self.solution(self.limit, self.limit), True, """Should be True."""
        )
        self.assertEqual(
            self.solution(self.limit,1), False, """Should be True."""
        )
        self.assertEqual(
            self.solution(self.limit,0.1), False, """Should be False."""
        )


In [30]:
"""@TriangleMaxArea"""
def triangle_limit(base, height):
    """This is my solution."""
    return (base * height) / 2 >= 10

In [31]:
%%testing TriangleMaxArea
nbtest_cases = [TriangleMaxArea]

### Creating a Variant 

When a question class is parameterize you can use the `variant` class function to create variants of the question:


In [32]:
variant = TriangleMaxArea.variant(
    name = "triangle_max_100",
    base = "b",
    height = "h",
    limit = 100, 
)
Markdown(variant.question())



Write a function called triangle_max_100 that takes two arguments, b and h. Calculate the area of a triangle with b and h. 
If the area of the triangle is greater than or equal to 100, return `True` otherwise return `False`. 


Add the tag `@TriangleMaxAreacc4e` to the docstring in your solution cell.


In [33]:
"""@TriangleMaxAreacc4e"""

def triangle_max_100(b, h):
    """Some question"""
    return (b * h) / 2 >= 100

In [34]:
%%testing variant
nbtest_cases = [variant]

## A Question with a Function and a Cell Variant

This question can be asked in function and cell-based forms.

In [35]:
import random 

from nbquiz import  FunctionQuestion

class TriangleArea(FunctionQuestion):
    """
    Write a function called `{name}` that takes two arguments, `{base}` and `{height}`. The function returns the area
    of a triangle that has the given `{base}` and `{height}`.
    """

    name = "triangle_area"
    base = "base"
    height = "height"
    
    annotations = {"base": float, "height": float, "return": float}

    def test_1_triangle_area(self):
        """Testing triangle_area()"""
        b = random.uniform(0, 100)
        h = random.uniform(0, 100)
        self.assertAlmostEqual(self.solution(b, h), (b * h) / 2)

In [36]:
from IPython.display import Markdown
Markdown(TriangleArea.question())



Write a function called `triangle_area` that takes two arguments, `base` and `height`. The function returns the area
of a triangle that has the given `base` and `height`.


Add the tag `@TriangleArea` to the docstring in your solution cell.


In [37]:
"""@TriangleArea"""

from numbers import Number

def triangle_area(base: Number, height: Number) -> Number:
    """The solution"""
    return (base * height) / 2

In [38]:
%%testing TriangleArea
nbtest_cases = [TriangleArea]

In [39]:
nbtest.assert_ok()

### A Cell Variant 

A question that inherits from a FunctionQuestion and a CellQuestion can be used to test a cell using the `run()` method of a `CellCacheEntry`.  

In [40]:
from nbquiz import CellQuestion

class TriangleAreaCell(TriangleArea, CellQuestion):
    """
    Create a cell that defines the variables `{base}` and `{height}`. The cell uses the variables to compute the area of a triangle.
    The cell should result in the computation.
    """


In [41]:
Markdown(TriangleAreaCell.question())



Create a cell that defines the variables `base` and `height`. The cell uses the variables to compute the area of a triangle.
The cell should result in the computation.


Add the tag `@TriangleAreaCell` to the docstring in your solution cell.


In [42]:
"""@TriangleAreaCell"""

base = 100
height = 200
area = (base * height) / 2

area

10000.0

In [43]:
%%testing TriangleAreaCell
nbtest_cases = [TriangleAreaCell]

In [44]:
nbtest.assert_ok()

In [45]:
"""@TriangleAreaCell"""

# incorrect calculation 

base = 100
height = 200
area = (base * height) / 4

area

5000.0

In [46]:
%%testing TriangleAreaCell
nbtest_cases = [TriangleAreaCell]

In [47]:
nbtest.assert_error()